In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import zenhan
from jeraconv import jeraconv
j2w = jeraconv.J2W()

In [ ]:
FILE_LIST = [
    "./rawdata/r2kentou-gyoukaku04.xlsx",
    "./rawdata/r2kentou-kisei05.xlsx",
    "./rawdata/r2teian-gyoukaku07.xlsx",
    "./rawdata/r2teian-kisei07.xlsx"
]
JSON_SUFFIX = "20210118" 

In [ ]:
df = pd.read_excel(FILE_LIST[0], header=[0,1])
df.columns = ["_".join(x) if x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df.columns.ravel()]
df = df.dropna(subset=["番号"])

In [ ]:
def conv_date(df, column):
    tmp = df[column].apply(lambda x: str(x).replace('\n','')).str.split('年', expand=True)
    tmp.rename(columns={0: 'Y', 1: 'MD'}, inplace=True)
    tmp['MD'] = tmp['MD'].apply(lambda x: zenhan.z2h(str(x)) if x is not None else '')
    tmp['Y'] = tmp['Y'].apply(lambda x: str(j2w.convert('令和' + x + '年')) + '年' if x != 'nan' else '')
    tmp['YMD'] = tmp['Y'] + tmp['MD']
    return tmp['YMD'].apply(lambda x: pd.to_datetime(x, format="%Y年%m月%d日") if x != '' else '')

In [ ]:
df['所管省庁への検討要請日'] = conv_date(df, '所管省庁への検討要請日')
df['回答取りまとめ日'] = conv_date(df, '回答取りまとめ日')

In [ ]:
df2 = pd.read_excel(FILE_LIST[1], header=[0,1])
df2.columns = ["_".join(x) if x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]
df2 = df2.dropna(subset=["番号"])
df2['所管省庁への検討要請日'] = conv_date(df2, '所管省庁への検討要請日')
df2['回答取りまとめ日'] = conv_date(df2, '回答取りまとめ日')

In [ ]:
df2.head()

In [ ]:
df3 = pd.read_excel(FILE_LIST[2], header=2)
df3.rename(columns={'所管省庁への\u3000\u3000検討要請日': '所管省庁への検討要請日', '提案事項名\n（タイトル）':'提案事項名', '提案主体名\n（会社名・団体名）':'提案主体名'}, inplace=True)
df3['所管省庁への検討要請日'] = conv_date(df3, '所管省庁への検討要請日')
df3['回答取りまとめ日'] = conv_date(df3, '回答取りまとめ日')

In [ ]:
df3.tail()

In [ ]:
df4 = pd.read_excel(FILE_LIST[3], header=3)
df4.rename(columns={'所管省庁への\u3000\u3000検討要請日': '所管省庁への検討要請日', '内閣府での\n回答取りまとめ\n日':'回答取りまとめ日', '提案事項名\n（タイトル）':'提案事項名', '提案主体名\n（会社名・団体名）':'提案主体名'}, inplace=True)
df4['所管省庁への検討要請日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df4['所管省庁への検討要請日'],'D')
df4['回答取りまとめ日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df4['回答取りまとめ日'],'D')

In [ ]:
df4.tail()

In [ ]:
df_gyosei = pd.merge(df4, df2, how='left', on='番号').drop(columns=[
    '提案事項','提案主体','所管省庁', #'回答参照先'
    'ワーキング・グループにおける\n処理方針'])
def a(x):
    try:
        tmp = x['所管省庁への検討要請日_y']
        if tmp is pd.NaT:
            tmp = x['所管省庁への検討要請日_x']
    except:
        print(x)
    return tmp
df_gyosei['所管省庁への検討要請日_y'] = df_gyosei.apply(a, raw=False, axis=1)
def b(x):
    try:
        tmp = x['回答取りまとめ日_x']
        if tmp == pd.NaT:
            tmp = x['回答取りまとめ日_y']
    except:
        print(x)
    return tmp
df_gyosei['回答取りまとめ日_x'] = df_gyosei.apply(b, raw=False, axis=1)
df_gyosei = df_gyosei.drop(columns=['所管省庁への検討要請日_x', '回答取りまとめ日_y'])
df_gyosei.rename(columns={
    '所管省庁への検討要請日_y': '所管省庁への検討要請日',
    '回答取りまとめ日_x': '回答取りまとめ日',
    '提案事項名': '提案事項',
    '提案主体名': '提案主体',
    '所管省庁_x': '所管省庁',
    '制度の所管省庁': '所管省庁',
    '備考_y': 'ワーキング・グループにおける処理方針'
}
, inplace=True)
df_gyosei['区分']='行政改革'
df_gyosei['通番']=df_gyosei['番号'].map('行-{:05d}'.format)
df_gyosei.head()

In [ ]:
df_kisei = pd.merge(df3, df, how='left', on='番号').drop(columns=[
    '提案事項','提案主体','所管省庁_y','備考_x'])
df_kisei['所管省庁への検討要請日_y'] = df_kisei.apply(a, raw=False, axis=1)
df_kisei['回答取りまとめ日_x'] = df_kisei.apply(b, raw=False, axis=1)
df_kisei = df_kisei.drop(columns=['所管省庁への検討要請日_x', '回答取りまとめ日_y'])
df_kisei.rename(columns={
    '所管省庁への検討要請日_y': '所管省庁への検討要請日',
    '回答取りまとめ日_x': '回答取りまとめ日',
    '提案事項名': '提案事項',
    '提案主体名': '提案主体',
    '所管省庁_x': '所管省庁',
    '備考_y': 'ワーキング・グループにおける処理方針'
}
, inplace=True)
df_kisei['区分']='規制改革'
df_kisei['通番']=df_kisei['番号'].map('規-{:05d}'.format)
df_kisei.head()

In [ ]:
df_all = pd.concat([df_gyosei,df_kisei], ignore_index=True)

## タグ付処理

In [ ]:
import MeCab
tagger = MeCab.Tagger("-Ochasen -u ./dict/local_dict.dic")

In [ ]:
def get_word_bag(text):
    try:
        tag = tagger.parseToNode(text)
        words = set()
        while tag:
            feature = tag.feature.split(",")
            if feature[0] == "名詞" and len(tag.surface) >= 2:
                words.add(tag.surface)
            tag = tag.next
        return list(words)
    except:
        return []

In [ ]:
df_all.columns

In [ ]:
all_text = df_all['提案事項'] + \
           df_all['提案の具体的内容'].replace(np.nan, '', regex=True) + \
           df_all['提案理由'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_制度の現状'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_対応の概要'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_該当法令等'].replace(np.nan, '', regex=True)
tmp = all_text.apply(get_word_bag)

In [ ]:
import json

with open('./dict/tag.json', encoding='utf-8') as json_file:
    rules_json = json.load(json_file)

def get_summary_tag(words):
    tags = []
    rules = rules_json['rules']

    for rule in rules:
        if any(item in words for item in rule['def']):
            tags.append(rule['tag'])
    return '\n'.join(map(str,tags))

In [ ]:
df_all['タグ'] = tmp.apply(get_summary_tag)

## データ保存

In [ ]:
# 概要
df_all['UID'] = df_all.index.map('UID{:05d}'.format)
df_all.drop_duplicates(subset=['通番'], inplace=True)
df_all[['UID','番号', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考', '所管省庁への検討要請日','所管省庁の検討結果_対応の分類',
        'ワーキング・グループにおける処理方針', '区分', '通番','タグ']].to_json('./s3static/src/assets/data{}.json'.format(JSON_SUFFIX), orient='records')
# 詳細
df_detailed = df_all[['UID','提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要']]
df_detailed = df_detailed.set_index('UID')
df_detailed.dropna(subset=['提案の具体的内容'], inplace=True) 
#df_detailed.drop_duplicates(inplace=True)     
df_detailed.to_json('./s3static/public/data/data{}.json'.format(JSON_SUFFIX), orient='index')

In [ ]:
df_all